In [30]:
# Colab Integration and Setup
### IF YOU ARE USING COLAB, UNCOMMENT AND RUN THIS BLOCK FIRST ###

# Mount google drive to allow access to your files
from google.colab import drive
drive.mount('/content/drive')

drive_folder = '/content/drive/MyDrive'

# Adjust this line to be the assignment1 folder in your google drive
notebook_folder = drive_folder + '/neuralEvolution'
%cd {notebook_folder}


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1j8nqeuQvJGhEynUrPzMKgTDtOKVmvmqN/neuralEvolution


This script performs evolutionary neural architecture search using a combination of genetic algorithms and lexicase selection to evolve Convolutional Neural Network (CNN) architectures for the CIFAR-10 dataset. The goal is to optimize multiple objectives—accuracy, model complexity, and training time—through an iterative process.

1. Environment Setup
The first block mounts Google Drive to Google Colab, allowing access to files stored in the drive. It then navigates to the specific folder containing the project files and installs necessary libraries.

In [31]:

%cd datasets
!bash get_datasets.sh
%cd ..

/content/drive/.shortcut-targets-by-id/1j8nqeuQvJGhEynUrPzMKgTDtOKVmvmqN/neuralEvolution/datasets
--2024-10-30 02:34:46--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  39.2MB/s    in 4.6s    

2024-10-30 02:34:50 (35.4 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
/content/drive/.shortcut-targets-by-id/1j8nqeuQvJGhEynUrPzMKgTDtOKVmvmqN/neuralEvolution


Data Preparation

The CIFAR-10 dataset, consisting of 60,000 images across 10 classes, is loaded and preprocessed.

Normalization: Converts pixel values to the range [0, 1] for stable training.

One-Hot Encoding: Converts labels to a format suitable for classification.

In [32]:
!pip install deap keras tensorflow matplotlib


In [34]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input
from keras.datasets import cifar10
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [35]:
# Step 1: Setup Environment
# Define the multi-objective fitness function to maximize accuracy, minimize complexity, and training time
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, -1.0))  # Maximize accuracy, minimize complexity, minimize time
creator.create("Individual", list, fitness=creator.FitnessMulti)



In [36]:
# Step 2: Data Preparation
# Load CIFAR-10 Dataset and preprocess
print("Loading CIFAR-10 dataset...")
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]
input_shape = x_train.shape[1:]
print("Dataset loaded. Number of classes:", num_classes)



Loading CIFAR-10 dataset...
Dataset loaded. Number of classes: 10


In [37]:
# Split training data into training and validation sets
val_split = 0.1
val_size = int(len(x_train) * val_split)
x_val, y_val = x_train[:val_size], y_train[:val_size]
x_train, y_train = x_train[val_size:], y_train[val_size:]

In [38]:
# Define the toolbox for evolutionary steps
toolbox = base.Toolbox()

In [39]:

# Step 3: Initial Population Generation
# Function to generate individuals with randomly defined layer types and configurations
print("Generating initial population...")
def generate_individual():
    layers = []
    num_layers = random.randint(2, 5)  # Random number of layers
    for _ in range(num_layers):
        layer_type = random.choice(['conv', 'dense'])
        if layer_type == 'conv':
            layers.append({
                'type': 'conv',
                'filters': random.choice([16, 32, 64, 128]),
                'activation': random.choice(['relu', 'tanh'])
            })
        elif layer_type == 'dense':
            layers.append({
                'type': 'dense',
                'units': random.choice([64, 128, 256, 512]),
                'activation': random.choice(['relu', 'tanh'])
            })
    print("Generated individual architecture:", layers)
    return layers

toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


Generating initial population...


Initial Population Generation

A function generates individuals (network architectures) with different configurations for the population.


Layer Variety: Each individual contains a random number of layers (2-5), which may be convolutional or fully connected (dense).


Random Parameters: Randomly assigns the number of filters/units and activation functions, resulting in diverse architectures for the population.


In [40]:
# Step 4: Implement Evaluation Function
# Create a neural network model from an individual representation


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D

# Create a neural network model from an individual representation
def create_cnn(architecture):
    model = Sequential()

    # Add Input layer first
    model.add(Input(shape=input_shape))

    added_flatten = False  # Track if a Flatten layer has been added

    for layer in architecture:
        if layer['type'] == 'conv' and not added_flatten:
            # Use 'same' padding to prevent dimensions from shrinking too quickly
            model.add(Conv2D(layer['filters'], (3, 3), activation=layer['activation'], padding='same'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
        elif layer['type'] == 'dense':
            if not added_flatten:
                model.add(Flatten())
                added_flatten = True
            model.add(Dense(layer['units'], activation=layer['activation']))

    # Ensure the model has been flattened before adding the final dense layer
    if not added_flatten:
        model.add(Flatten())

    # Add the final output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    print("Model compiled with architecture:")
    model.summary()

    return model


In [41]:

# Evaluation function to assess the fitness of an individual
import time

def evaluate_cnn(individual):
    print("Evaluating individual:", individual)
    model = create_cnn(individual)
    start_time = time.time()
    # Train the model on a small subset for quicker evaluation
    history = model.fit(x_train[:2000], y_train[:2000], validation_data=(x_val, y_val), epochs=3, batch_size=32, verbose=1)
    accuracy = model.evaluate(x_val, y_val, verbose=1)[1]
    model_complexity = model.count_params()
    training_time = time.time() - start_time
    print(f"Accuracy: {accuracy}, Model Complexity: {model_complexity}, Training Time: {training_time}")
    return accuracy, model_complexity, training_time

The evaluate_cnn() function creates and trains a CNN model based on the individual's architecture. It calculates a fitness score based on multiple criteria: accuracy, model complexity, and training time.


Model Creation: Constructs a CNN model with layers defined by the individual’s architecture.

Model Training: Uses a subset of the CIFAR-10 dataset to speed up the evaluation.

Fitness Metrics: Returns accuracy, model complexity (number of parameters), and a placeholder training time.

In [42]:
# Step 5: Selection Process
# Lexicase selection function to select individuals based on multiple objectives
def lexicase_selection(population, k):
    selected = []
    for _ in range(k):
        candidates = population[:]
        random.shuffle(candidates)
        objectives = list(range(len(population[0].fitness.values)))
        random.shuffle(objectives)
        for obj in objectives:
            best_candidates = [c for c in candidates if c.fitness.values[obj] == max(c.fitness.values[obj] for c in candidates)]
            candidates = best_candidates
            if len(candidates) == 1:
                break
        selected.append(candidates[0])
    return selected



Selection Process

The script uses both tournament selection and lexicase selection to enhance the evolutionary process.

Tournament Selection: Used in the early stages of evolution.

Lexicase Selection: Used in later generations to select individuals that excel in multiple objectives (accuracy, complexity, and training time). It maintains diversity by selecting candidates based on different combinations of objectives, ensuring robust and well-rounded architectures.

In [43]:
# Register genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select_tournament", tools.selTournament, tournsize=3)
toolbox.register("select_lexicase", lexicase_selection, k=10)
toolbox.register("evaluate", evaluate_cnn)

In [44]:
# Step 6: Crossover and Mutation
# Apply crossover and mutation to maintain diversity in population
def apply_crossover_and_mutation(offspring, cxpb, mutpb):
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values



Crossover and Mutation

To maintain architectural diversity, crossover and mutation are applied to offspring.


Crossover: cxTwoPoint crossover is used to exchange genetic information between individuals.

Mutation: mutShuffleIndexes mutation is used to randomly change individual architectures, promoting exploration of the search space.


In [45]:
# Step 7: Iterative Evolution Process
# Main function to run the evolutionary algorithm
def main():
    population = toolbox.population(n=20)
    NGEN = 10
    CXPB, MUTPB = 0.5, 0.2
    K = 10  # Define the value for k explicitly

    # Log progress of evolution
    fitness_log = []

    for gen in range(NGEN):
        print(f"-- Generation {gen} --")
        # Evaluate all individuals in the population
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        # Log the average fitness of the current generation
        avg_fitness = np.mean([ind.fitness.values[0] for ind in population])
        fitness_log.append(avg_fitness)
        print(f"Average Fitness: {avg_fitness}")

        # Step 8: Hybrid Selection Process
        # Use tournament selection for early generations and lexicase selection for later generations
        if gen < NGEN // 2:
            offspring = toolbox.select_tournament(population, len(population))
        else:
            #offspring = toolbox.select_lexicase(population, len(population))
            #offspring = toolbox.select_lexicase(population)
            offspring = toolbox.select_lexicase(population, K)  # Pass k explicitly

        offspring = list(map(toolbox.clone, offspring))

        # Step 6: Apply Crossover and Mutation
        apply_crossover_and_mutation(offspring, CXPB, MUTPB)

        # Replace population with the offspring
        population[:] = offspring

    # Plot fitness progression
    plt.plot(range(NGEN), fitness_log)
    plt.xlabel('Generation')
    plt.ylabel('Average Fitness')
    plt.title('Evolutionary Progress')
    plt.show()

     # Step 10: Final Evaluation
    # Print the final population's fitness values
    for ind in population:
        print(ind.fitness.values)

    # Gather and return the best individual
    best_ind = tools.selBest(population, 1)[0]
    print(f"Best individual: {best_ind}")
    return best_ind



Iterative Evolution Process
The main() function handles the iterative process for evolving the population.

Population Initialization: Creates an initial population of 20 individuals.

Generational Loop: Evolves the population over 10 generations:

Evaluation: All individuals are evaluated, and their fitness values are assigned.

Logging: Average fitness for each generation is recorded and plotted to visualize the evolutionary progress.

Hybrid Selection: Uses tournament selection for early generations (for quick exploration) and lexicase selection for later generations (to focus on multiple objectives).

Crossover and Mutation: Ensures diversity by modifying offspring before replacing the old population.

Final Evaluation: The best individual is selected and returned as the best architecture found during the evolution process.

In [ ]:


if __name__ == "__main__":
    best_architecture = main()
    print("Best architecture found:", best_architecture)


Generated individual architecture: [{'type': 'conv', 'filters': 16, 'activation': 'tanh'}, {'type': 'conv', 'filters': 128, 'activation': 'relu'}]
Generated individual architecture: [{'type': 'dense', 'units': 256, 'activation': 'relu'}, {'type': 'dense', 'units': 512, 'activation': 'relu'}]
Generated individual architecture: [{'type': 'conv', 'filters': 32, 'activation': 'relu'}, {'type': 'conv', 'filters': 16, 'activation': 'relu'}, {'type': 'dense', 'units': 256, 'activation': 'tanh'}]
Generated individual architecture: [{'type': 'dense', 'units': 256, 'activation': 'relu'}, {'type': 'conv', 'filters': 128, 'activation': 'tanh'}, {'type': 'conv', 'filters': 128, 'activation': 'tanh'}, {'type': 'dense', 'units': 64, 'activation': 'tanh'}]
Generated individual architecture: [{'type': 'dense', 'units': 512, 'activation': 'tanh'}, {'type': 'conv', 'filters': 128, 'activation': 'tanh'}, {'type': 'dense', 'units': 128, 'activation': 'relu'}]
Generated individual architecture: [{'type': 'd

Model: "sequential_298"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_401 (Conv2D)                  │ (None, 32, 32, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_401 (MaxPooling2D)     │ (None, 16, 16, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_402 (Conv2D)                  │ (None, 16, 16, 128)         │          18,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_402 (MaxPooling2D)     │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_298 (Flatten)                │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_603 (Dense)                    │ (None, 10)                  │          81,930 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 100,938 (394.29 KB)

 Trainable params: 100,938 (394.29 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_544 (Conv2D)                  │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_544 (MaxPooling2D)     │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_545 (Conv2D)                  │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_545 (MaxPooling2D)     │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_384 (Flatten)                │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_756 (Dense)                    │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_757 (Dense)                    │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,275,594 (16.31 MB)

 Trainable params: 4,275,594 (16.31 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
48/63 ━━━━━━━━━━━━━━━━━━━━ 4s 268ms/step - accuracy: 0.1188 - loss: 2.6606

Key Features of This Implementation

Diverse Architectures: Each individual contains a different combination of layers, ensuring a broad search space and better exploration.

Adaptive Selection: Uses a combination of tournament and lexicase selection, leading to a balance between exploration and exploitation. Early generations focus on exploring different architectures, while later generations aim for well-rounded performance across multiple objectives.

Practical Training Strategy: Uses a subset of the CIFAR-10 dataset to speed up fitness evaluation, making the evolution process computationally feasible.

Multi-Objective Optimization: Balances between maximizing accuracy and minimizing model complexity and training time, leading to efficient, high-performing architectures.

Visualization: Tracks average fitness across generations to provide insight into the evolutionary process.


 Modifications to Incorporate the Improvements
references:


https://github.com/harvitronix/neural-network-genetic-algorithm/tree/master
https://blog.coast.ai/lets-evolve-a-neural-network-with-a-genetic-algorithm-code-included-8809bece164
https://github.com/jliphard/DeepEvolve
https://lethain.com/genetic-algorithms-cool-name-damn-simple/


Enhanced Crossover and Mutation:

Implement a more sophisticated crossover function that can swap convolutional and dense blocks between individuals instead of the entire architecture.
Introduce adaptive mutation rates: Increase mutation rates if the average fitness across generations is not improving.
Elitism:

Preserve the top 2 or 3 individuals unmodified between generations.
Adjust the existing population update to include both lexicase-selected individuals and elites.
Expanded Search Space for Hyperparameters:

Add more hyperparameters for evolution such as learning rate, batch size, and dropout rates.
Modify the generate_individual() function to include these additional parameters.
Hybrid Fitness Function:

Update evaluate_cnn() to compute a hybrid fitness score including:
Accuracy.
Model complexity.
Training time.
Normalize these values and use them collectively during lexicase selection.
Component-Level Evolution:

Split architectures into smaller modules (e.g., convolutional blocks, dense layers).
Evolve these modules separately before using them to build complete architectures, mimicking a simplified blueprint approach.
Logging and Visualization:

Add detailed logs for each individual’s training progress.
Plot the fitness trend across all generations for each objective—accuracy, complexity, and training time—to visualize trade-offs.